# tf.dataを使ってCSVをロードする

CSVデータを`tf.data.Dataset`にロードする方法

このチュートリアルでは、タイタニック号の乗客リストから取られたデータを使用する。
乗客が生き残る可能性を、年齢、性別、チケットの等級、そして乗客が一人で旅行しているか否かといった特性から予測することを試みる。

In [29]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [30]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [31]:
# numpyの値を読みやすくする
np.set_printoptions(precision=3, suppress=True)

## データのロード

In [32]:
!head {train_file_path}

'head' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


In [33]:
# 入力ファイル中のCSV列
with open(train_file_path, 'r') as f:
    names_row = f.readline()

CSV_COLUMNS = names_row.rstrip('\n').split(',')
print(CSV_COLUMNS)

['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']


データセットコンストラクタはこれらのラベルを自動的にピックアップする。

使用するファイルの1行目に列名がない場合、`make_csv_dataset`関数の`column_names`引数に文字列のリストとして渡すこと。

In [34]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

dataset = tf.data.experimental.make_csv_dataset(
     ...,
     column_names=CSV_COLUMNS,
     ...)

SyntaxError: positional argument follows keyword argument (<ipython-input-34-3ebdb4af0a80>, line 6)

もしデータセットから一部の列を除きたい場合には、使用したい列だけを含むリストを作り、コンストラクタのオプションである`select_columns` 引数として渡すこと。

In [35]:
drop_columns = ["fare", "embark_town"]
columns_to_use = [col for col in CSV_COLUMNS if col not in drop_columns]

dataset = tf.data.experimental.make_csv_dataset(
    ...,
    select_columns=columns_to_use
    ...
)

SyntaxError: invalid syntax (<ipython-input-35-2604f64009a5>, line 7)

各サンプルのラベルとなる列を特定し、それが何であるかを示す必要がある。
つまり、モデルに推論させたい列を指定する必要がある。

In [36]:
LABLES = [0, 1]
LABEL_COLUMN = "survived"

FEATURE_COLUMNS = [column for column in CSV_COLUMNS if column != LABEL_COLUMN]

コンストラクタの引数の値が揃ったので、ファイルからCSVデータを読み込んでデータセットを作る。

In [37]:
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name=LABEL_COLUMN,
        na_value='?',
        num_epochs=1,
        ignore_errors=True
    )
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

データセットを構成する要素は、(複数のサンプル, 複数のラベル)の形のタプルとして表されるバッチになっている。
サンプル中のデータは(行ベースのテンソルではなく)列ベースのテンソルとして構成され、それぞれはバッチサイズ(このケースでは12個)の要素が含まれる。

In [38]:
examples, labels = next(iter(raw_train_data))
print("EXAMPLES: \n", examples, '\n')
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'female', b'male', b'male', b'male', b'female', b'male', b'male',
       b'female', b'male', b'female', b'male', b'female'], dtype=object)>), ('age', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([38.  , 28.  , 48.  , 19.  , 58.  , 28.  , 28.  , 28.  , 43.  ,
       14.  ,  0.83, 24.  ], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([1, 8, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])>), ('parch', <tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0])>), ('fare', <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([ 71.283,  69.55 ,   7.854,  10.171, 146.521,   7.725,   8.05 ,
         7.75 ,   8.05 ,  30.071,  29.   ,  13.   ], dtype=float32)>), ('class', <tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'First', b'Third', b'Third', b'Third', b'First', b'Third',
       b'Third', b'Third', b'Third', b'Second', b'Second', b'Sec

## データの前処理

### カテゴリデータ

このCSVデータ中のいくつかの列はカテゴリ列になっている。つまり、その中身は、限られた選択肢の中のひとつである必要がある。

このCSVでは、これらの選択肢はテキストとして表現されている。このテキストは、モデルの訓練を行えるように数字に変換する必要がある。これをやりやすくするため、カテゴリ列のリストとその選択肢のリストを作成する必要がある。

In [39]:
CATEGORIES = {
    "sex": ["male", "female"],
    "class": ["First", "Second", "Third"],
    "deck": ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    "embark_town": ["Cherbourg", "Southhampton", "Queenstown"],
    "alone": ['y', 'n']
}

カテゴリ値のテンソルを受け取り、それを名前のリストとマッチングして、さらにワンホット・エンコーディングを行う関数を書く

In [40]:
def process_categorical_data(data, categories):
    """カテゴリ値を表すワンホット・エンコーディングされたテンソルを返す"""

    # 最初の' 'を取り除く
    data = tf.strings.regex_replace(data, '^ ', '')

    # 最後の'.'を取り除く
    data = tf.strings.regex_replace(data, r'\.$', '')

    # ワンホット・エンコーディング
    # dataを1次元(リスト)から2次元(要素が1個のリストのリスト)にreshape
    data = tf.reshape(data, [-1, 1])
    # それぞれの要素について、カテゴリ数の長さの真偽値のリストで、
    # 要素のカテゴリのラベルが一致したところがTrueとなるものを作成
    data = categories == data
    # 真偽値を浮動小数点数にキャスト
    data = tf.cast(data, tf.float32)

    # エンコーディング全体を次の1行に収めることもできる。
    # data = tf.cast(categories == tf.reshape(data, [-1, 1]), tf.float32)
    return data

この処理を可視化するため、最初のバッチからカテゴリ列のテンソル1つを取り出し、処理を行い、前後の状態を示す。

In [41]:
class_tensor = examples["class"]
class_tensor

<tf.Tensor: shape=(12,), dtype=string, numpy=
array([b'First', b'Third', b'Third', b'Third', b'First', b'Third',
       b'Third', b'Third', b'Third', b'Second', b'Second', b'Second'],
      dtype=object)>

In [42]:
class_categories = CATEGORIES["class"]
class_categories

['First', 'Second', 'Third']

In [43]:
processed_class = process_categorical_data(class_tensor, class_categories)
processed_class

<tf.Tensor: shape=(12, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)>

In [44]:
print("Size of batch: ", len(class_tensor.numpy()))
print("Number of category labels: ", len(class_categories))
print("Shape of one-hot encoded tensor: ", processed_class.shape)

Size of batch:  12
Number of category labels:  3
Shape of one-hot encoded tensor:  (12, 3)


### 連続データ

連続データは値が0と1の間になるように標準化する必要がある。これを行うために、それぞれの値を、1を列値の平均の2倍で割ったものを掛ける関数を書く。データの2次元のテンソルへのreshapeも行う。

In [45]:
def process_continuous_data(data, mean):
    # dataの標準化
    data = tf.cast(data, tf.float32) * 1/(2*mean)
    return tf.reshape(data, [-1, 1])

この計算を行うためには、列値の平均が必要。今回のチュートリアルでは平均値が下記の通り得られたものとする。

In [46]:
MEANS = {
    "age": 29.631308,
    "n_siblings_spouses": 0.545455,
    "parch": 0.379585,
    "fare": 34.385399
}

In [47]:
age_tensor = examples["age"]
age_tensor

<tf.Tensor: shape=(12,), dtype=float32, numpy=
array([38.  , 28.  , 48.  , 19.  , 58.  , 28.  , 28.  , 28.  , 43.  ,
       14.  ,  0.83, 24.  ], dtype=float32)>

In [48]:
process_continuous_data(age_tensor, MEANS["age"])

<tf.Tensor: shape=(12, 1), dtype=float32, numpy=
array([[0.641],
       [0.472],
       [0.81 ],
       [0.321],
       [0.979],
       [0.472],
       [0.472],
       [0.472],
       [0.726],
       [0.236],
       [0.014],
       [0.405]], dtype=float32)>

### データの前処理

これらの前処理のタスクを1つの関数にまとめ、データセット内のバッチにマッピングできるようにする。

In [49]:
def preprocess(features, labels):
    # カテゴリ特徴量の処理
    for feature in CATEGORIES.keys():
        features[feature] = process_categorical_data(features[feature],
                                                        CATEGORIES[feature])
    # 連続特徴量の処理
    for feature in MEANS.keys():
        features[feature] = process_continuous_data(features[feature],
                                                       MEANS[feature])

    # 特徴量を1つのテンソルに組み立てる
    features = tf.concat([features[column] for column in FEATURE_COLUMNS], 1)

    return features, labels

次に、`tf.Dataset.map`関数を使って適用し、過学習を防ぐためにデータセットをシャッフルする。

In [50]:
train_data = raw_train_data.map(preprocess).shuffle(500)
test_data = raw_test_data.map(preprocess)

In [51]:
examples, labels = next(iter(train_data))
examples, labels

(<tf.Tensor: shape=(12, 24), dtype=float32, numpy=
 array([[1.   , 0.   , 0.472, 0.   , 0.   , 0.103, 0.   , 0.   , 1.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 1.   , 0.   ],
        [0.   , 1.   , 0.523, 0.   , 0.   , 0.114, 0.   , 0.   , 1.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 1.   , 0.   ],
        [1.   , 0.   , 0.472, 0.   , 0.   , 0.115, 0.   , 0.   , 1.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 1.   , 0.   ],
        [1.   , 0.   , 0.253, 0.917, 1.317, 0.105, 0.   , 0.   , 1.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 1.   , 0.   , 0.   , 0.   , 1.   ],
        [1.   , 0.   , 0.776, 0.   , 0.   , 1.152, 1.   , 0.   , 0.   ,
         0.   , 1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 1.   

## モデルの構築

この例では、Keras Functional APIを使用し、単純なモデルを構築するために`get_model`コンストラクタでラッピングしている。

In [52]:
def get_model(input_dim, hidden_units=[100]):
    """複数の層を持つ Keras モデルを作成

    引数:
        input_dim: (int) バッチ中のアイテムの形状
        labels_dim: (int) ラベルの形状
        hidden_units: [int] DNN の層のサイズ（入力層が先）
        learning_rate: (float) オプティマイザの学習率
    戻り値:
        Keras モデル
    """
    inputs = tf.keras.Input(shape=(input_dim,))
    x = inputs

    for units in hidden_units:
        x = tf.keras.layers.Dense(units, activation="relu")(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs, outputs)

    return model

In [53]:
input_shape, output_shape = train_data.output_shapes
input_dimension = input_shape.dims[1] # [0]はバッチサイズ

## 訓練、評価、そして予測

In [54]:
model = get_model(input_dimension)
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)
model.fit(train_data, epochs=20)

Epoch 1/20
53/53 [==============================] - 1s 12ms/step - loss: 0.6026 - accuracy: 0.7193
Epoch 2/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4999 - accuracy: 0.7847
Epoch 3/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4533 - accuracy: 0.8086
Epoch 4/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4328 - accuracy: 0.8086
Epoch 5/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4235 - accuracy: 0.8198
Epoch 6/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4176 - accuracy: 0.8230
Epoch 7/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4148 - accuracy: 0.8246
Epoch 8/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4172 - accuracy: 0.8214
Epoch 9/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4048 - accuracy: 0.8309
Epoch 10/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4060 - accuracy: 0.8262
Epoch 11/20
53/53 

In [55]:
test_loss, test_accuracy = model.evaluate(test_data)
print("\n\nTest Loss {}, Test Accuracy {}".format(test_loss, test_accuracy))

22/Unknown - 0s 6ms/step - loss: 0.4395 - accuracy: 0.8068

Test Loss 0.43948988210071216, Test Accuracy 0.8068181872367859


単一のバッチ、または、バッチからなるデータセットのラベルを推論する場合には、`tf.keras.Model.predict`を使う

In [56]:
predictions = model.predict(test_data)

# 結果のいくつかを表示
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print("Predicted survival: {:.2%}".format(prediction[0]),
          " | Actual outcome: ",
          ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 90.75%  | Actual outcome:  DIED
Predicted survival: 44.78%  | Actual outcome:  SURVIVED
Predicted survival: 19.82%  | Actual outcome:  DIED
Predicted survival: 11.56%  | Actual outcome:  DIED
Predicted survival: 93.40%  | Actual outcome:  DIED
Predicted survival: 64.14%  | Actual outcome:  DIED
Predicted survival: 10.08%  | Actual outcome:  DIED
Predicted survival: 48.89%  | Actual outcome:  SURVIVED
Predicted survival: 11.22%  | Actual outcome:  SURVIVED
Predicted survival: 6.41%  | Actual outcome:  DIED
